In [17]:
import networkx as nx
import datasets

# add .. to pythonpath
import sys
sys.path.append('..')
from dataset_loaders import *
from utils import eg2nx

In [18]:
# cd to ..
import os
os.chdir('/Users/tscp/testdir/easygraph-bench/')
chem = eg2nx(load_cheminformatics())
chem

In [24]:
road = load_road()
road

In [20]:
road.to_directed()

In [26]:
# get the largest connected component of road
lcc_road = max(nx.connected_components(road), key=len)
lcc_road

{'117536',
 '86770',
 '106761',
 '49915',
 '125007',
 '76179',
 '97545',
 '91835',
 '1545',
 '117842',
 '41629',
 '69955',
 '68820',
 '67586',
 '42226',
 '70253',
 '88960',
 '107906',
 '32748',
 '59907',
 '92382',
 '22518',
 '64276',
 '126548',
 '63417',
 '114038',
 '29787',
 '88679',
 '111326',
 '121232',
 '13997',
 '57572',
 '29541',
 '112057',
 '80490',
 '68258',
 '16111',
 '119779',
 '92887',
 '55963',
 '37170',
 '81955',
 '40635',
 '57787',
 '7259',
 '54977',
 '34215',
 '85134',
 '9355',
 '45972',
 '58583',
 '51659',
 '29954',
 '125704',
 '84025',
 '104239',
 '122355',
 '65884',
 '11854',
 '23659',
 '80905',
 '24979',
 '35518',
 '6555',
 '53139',
 '86315',
 '30700',
 '4544',
 '41721',
 '91855',
 '58094',
 '107595',
 '90760',
 '15826',
 '19143',
 '111110',
 '49696',
 '6951',
 '53702',
 '88755',
 '107994',
 '99046',
 '27004',
 '62335',
 '113697',
 '47903',
 '87359',
 '12913',
 '120073',
 '37885',
 '94321',
 '59515',
 '6301',
 '42754',
 '74924',
 '100383',
 '100509',
 '86543',
 '1120

In [28]:
l = road.subgraph(lcc_road)
l, len(l.nodes), len(l.edges)

(<networkx.classes.graph.Graph at 0x16b5fa3b0>, 126146, 161950)

In [29]:
len(road.nodes)

129164